In [ ]:
import matplotlib.cm
import scipy.stats.mstats
%config InlineBackend.figure_formats = ['png']

import multiprocessing
import pandas as pd
from data import rciw_mjhd, rmad_hd, collect_energy_data_paths, get_project_and_benchmark_from_consumption_path, \
    get_normalized_mj_from_file, rmad_hd, custom_mquantiles_cimh_hd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import scipy.stats as stats


In [ ]:
ITERATIONS = 30
MAX_SAMPLES_PER_ITERATION = 300
SAMPLE_STEP = 25
SAMPLES_PER_ITERATION_RANGE = range(SAMPLE_STEP, MAX_SAMPLES_PER_ITERATION + 1, SAMPLE_STEP)

THRESHOLDS = [0.5, 1, 3, 5, 10]


In [ ]:
benchmarks = defaultdict(list)

for path in collect_energy_data_paths():
    (proj, bench) = get_project_and_benchmark_from_consumption_path(path)
    if proj == 'tracing':
        continue
    benchmarks[proj + '/' + bench.replace("\\", "/")].extend(get_normalized_mj_from_file(path).tolist())
# benchmarks


In [ ]:

max_samples = max(map(len, benchmarks.values()))
too_little_samples = [key for key, values in benchmarks.items() if len(values) != max_samples]
consumption = pd.DataFrame({k: v for k, v in benchmarks.items() if k not in too_little_samples})
# pd.DataFrame({'dropped': too_little_samples, 'samples': list(map(lambda id: len(benchmarks[id]), too_little_samples))})
consumption

In [ ]:
# sns.stripplot(consumption.T[(consumption.T.max(axis='columns') - consumption.T.min(axis='columns')) > 50][:200], size=1, jitter=False)
# consumption.columns[(consumption.max(axis='rows') - consumption.min(axis='rows')) > 50]
large_consumption_difference = consumption.T[
    (consumption.T.max(axis='columns') - consumption.T.min(axis='columns')) > 50]


In [ ]:
sns.lineplot(large_consumption_difference.T, markers=False, estimator=None, legend=False, alpha=0.3)


In [ ]:
# consumption # consumption in mJ per benchmark execution
print(f"Maximum energy consumption for a single execution within 100ms is {consumption.max().max():.5}mJ.")
print(
    f"This translates to at most {consumption.max().max() / 0.1 / 1000:.3}W. This is {'not ' if consumption.max().max() / 0.1 / 1000 > 11.65 else ''}within the limit of a single core of the tested CPU.")

In [ ]:

actual_instr = pd.read_csv("instructions.csv", sep=";", names=('project', 'file', 'id', 'instructions'))
actual_instr['id'] = actual_instr['project'] + '/' + actual_instr['id'].map(lambda s: s.strip())
actual_instr = actual_instr.drop(columns=['project', 'file']).set_index('id')
actual_instr = actual_instr.loc[actual_instr['instructions'] > 0]
actual_instr = actual_instr.loc[actual_instr['instructions'] < 4.2e9 / 10]
actual_instr


In [ ]:
instruction_consumption = actual_instr.join(consumption.T, how='inner')

In [ ]:
sns.scatterplot(x=instruction_consumption['instructions'],
                y=stats.mstats.hdmedian(instruction_consumption.drop(columns=["instructions"]), axis=1))
plt.ylabel("Energy (mJ)")
plt.xlabel("Executed instruction count")


In [ ]:
sns.histplot(actual_instr, log_scale=(10, False), legend=False)
plt.xlabel("Number of executed instructions")

In [ ]:
f, ax = plt.subplots(2)
# sns.histplot(consumption.to_numpy().flatten(), bins=30, log_scale=(10, False), ax=ax[0], legend=False, stat='density')
# sns.histplot(actual_instr, bins=30, ax=ax[1], log_scale=(10, False), legend=False, stat='density')

sns.kdeplot(np.ma.getdata(stats.mstats.hdmedian(instruction_consumption.drop(columns=["instructions"]), axis=1), ),
            log_scale=(10, False), ax=ax[0], legend=False)
ax[0].set_ylim([0, 0.25])
ax[0].set_title("Median_hj energy consumption (mJ) density")
sns.kdeplot(instruction_consumption['instructions'], ax=ax[1], log_scale=(10, False), legend=False, )
ax[1].set_title("Benchmark length (# exec. instr.) density ")
ax[1].set_ylim([0, 0.25])
f.tight_layout()
plt.show()

meds = np.ma.getdata(stats.mstats.hdmedian(instruction_consumption.drop(columns=["instructions"]), axis=1), )
print(np.corrcoef(meds, instruction_consumption["instructions"]))
print(stats.spearmanr(meds, instruction_consumption["instructions"]))

This Spearman Correlation statistic of 0.95 shows a high correlation between the two Ordinal variables. 
The p-value of 0.0 is lower than 0.05 and we reject the null hypothesis that the data is independent from each other. Which it isn't since it is based on the same code. This shows however that median energy and benchmark length have a correlation.

## Calculate Stability
Define a multithreadable task that can calculate a certain statistic over a multi-index range of up to iterations and up to samples.

We calculate both RCIW and RMAD, which are both defined in [data.py](./data.py).


In [ ]:
from typing import Callable, Tuple
from multiprocessing.pool import ThreadPool


def pooled_task(stat_fn: Callable[[np.ndarray], float], bench_id: str, values: np.ndarray, iterations: int,
                samples: int) -> Tuple[Tuple[str, int, int], float]:
    # indices = [i % 300 < samples for i in range(300 * iterations)]
    # creates a Basic Indexing view into values that has shape (30, -1) where -1 can be up to 300
    try:
        return ((bench_id, iterations, samples), stat_fn(np.array(values[:iterations, :samples].flatten())))
    except IndexError:
        return ((bench_id, iterations, samples), np.NaN)


pool = ThreadPool(processes=14)

In [ ]:
from scipy.stats.mstats import mquantiles_cimj


def calc_rciw(data):
    # print(len(data))
    return rciw_mjhd(data, 0.05)
    # diff = mquantiles_cimj(data, prob=[0.5])


try:
    rciw_data = pickle.load(open('rciw_95.pickle', 'rb'))
except FileNotFoundError:
    args = []
    for bench in consumption.columns:
        arranged = np.array(consumption[bench]).reshape(ITERATIONS, -1)
        for iterations in range(ITERATIONS):
            for samples in SAMPLES_PER_ITERATION_RANGE:
                args.append([calc_rciw, bench, arranged, iterations + 1, samples])

    rciw_data = {k: v for k, v in pool.starmap(pooled_task, args)}
    pickle.dump(rciw_data, open('rciw_95.pickle', 'wb'))

In [ ]:
def med_hj(data):
    return custom_mquantiles_cimh_hd(data, prob=[0.5])[0]


try:
    median_data = pickle.load(open('medians.pickle', 'rb'))
except FileNotFoundError:
    args = []
    for bench in consumption.columns:
        arranged = np.array(consumption[bench]).reshape(ITERATIONS, -1)
        for iterations in range(ITERATIONS):
            for samples in SAMPLES_PER_ITERATION_RANGE:
                args.append([med_hj, bench, arranged, iterations + 1, samples])

    median_data = {k: v for k, v in pool.starmap(pooled_task, args)}
    pickle.dump(median_data, open('medians.pickle', 'wb'))

In [ ]:
try:
    rmad_data = pickle.load(open('rmad.pickle', 'rb'))
except FileNotFoundError:
    args = []
    for bench in consumption.columns:
        arranged = np.array(consumption[bench]).reshape(ITERATIONS, -1)
        for iterations in range(ITERATIONS):
            for samples in SAMPLES_PER_ITERATION_RANGE:
                args.append([rmad_hd, bench, arranged, iterations + 1, samples])

    rmad_data = {k: v for k, v in pool.starmap(pooled_task, args)}
    pickle.dump(rmad_data, open('rmad.pickle', 'wb'))

In [ ]:
# rciw = pd.DataFrame(rciw_data)

rciw = pd.DataFrame(rciw_data.values(),
                    index=pd.MultiIndex.from_tuples(rciw_data.keys(), names=["bench", "iterations", "samples"]),
                    columns=['rciw'])
rmad = pd.DataFrame(rmad_data.values(),
                    index=pd.MultiIndex.from_tuples(rmad_data.keys(), names=["bench", "iterations", "samples"]),
                    columns=['rmad'])
medians = pd.DataFrame(rmad_data.values(),
                    index=pd.MultiIndex.from_tuples(median_data.keys(), names=["bench", "iterations", "samples"]),
                    columns=['median'])

stabilities = rmad.join(rciw)


In [ ]:
fig, ax = plt.subplots(1)

sns.kdeplot(stabilities * 100, ax=ax, log_scale=(10, False), common_norm=True, legend=True)
# sns.kdeplot(stabilities, x='rciw', ax=ax, log_scale=(10, False), legend=True)
plt.xlabel("Stability (%)")
ax.legend(['rmad', 'rciw'])


In [ ]:
# iters = range(1,31)
iters = range(5, 31, 5)
fig, axs = plt.subplots(2, 3, sharex=True, sharey=True)
# local = rciw.loc[(slice(None), i, slice(None))]

for idx, s in enumerate(iters):
    ax = axs.flatten()[idx]
    sns.lineplot(medians.loc[(slice(None), s, slice(None))], x='samples', y='median', units='bench', hue='bench',
                 legend=False, ax=ax, estimator=None)
    ax.set_title(f"{s} iterations")
    ax.set_xlabel("Samples per iteration")
    # ax.set_ylim([0, 1.5])
plt.tight_layout()
# plt.yscale('log')

## Classify stability 
For both stability measures, classify for each threshold what proportion is stable.

In [ ]:
def ratio_stable(data, x, y, subplot_value):
    return ((data * 100) < subplot_value).loc[slice(None), x, y].value_counts(normalize=True).get(True, 0)


In [ ]:
classified = dict()
for idx, threshold in enumerate(THRESHOLDS):

    values = []
    for iteration in range(ITERATIONS):
        for samples in SAMPLES_PER_ITERATION_RANGE:
            ratio = ratio_stable(rciw, iteration + 1, samples, threshold)
            values.append(ratio)
    reshaped = np.array(values).reshape(ITERATIONS, len(SAMPLES_PER_ITERATION_RANGE))
    classified[threshold] = reshaped

In [ ]:
classified_rmad = dict()
for idx, threshold in enumerate(THRESHOLDS):

    values = []
    for iteration in range(ITERATIONS):
        for samples in SAMPLES_PER_ITERATION_RANGE:
            ratio = ratio_stable(rmad, iteration + 1, samples, threshold)
            values.append(ratio)
    reshaped = np.array(values).reshape(ITERATIONS, len(SAMPLES_PER_ITERATION_RANGE))
    classified_rmad[threshold] = reshaped

In [ ]:
rmad_samples = rmad.reset_index()
rmad_samples['num_samples'] = rmad_samples['iterations'] * rmad_samples['samples']
rmad_samples = rmad_samples.sort_values(['rmad']).drop_duplicates(['bench', 'num_samples'], keep='last')
fig = plt.figure(dpi=300, figsize=(12, 4))
plt.xlabel("Total number of samples")
plt.ylabel("Proportion of stable samples")
for threshold in THRESHOLDS:
    local = rmad_samples.copy(deep=True)
    local['stable'] = local['rmad'] < (threshold / 100)
    percentage_stable = local.set_index('bench').groupby('num_samples')['stable'].agg(
        'mean')  # mean calculates ratio of True/(True+False)
    line = sns.lineplot(percentage_stable, ax=fig.gca(), label=str(threshold))
    line.get_legend().set_title("Threshold (%)")

In [ ]:
import matplotlib

normalize = matplotlib.colors.Normalize(0, 1)
cmap = 'PiYG'
color_scalar = matplotlib.cm.ScalarMappable(norm=normalize, cmap=cmap)


def multi_implot(data: dict):
    fig, axs = plt.subplots(1, len(data), figsize=(16, 10), dpi=600)

    for idx, (threshold, ratio) in enumerate(data.items()):
        ax = axs[idx]
        ax.imshow(ratio, cmap=cmap, interpolation='none')
        ax.set_label("{}%".format(threshold))
        ax.set_xticks(range(0, len(SAMPLES_PER_ITERATION_RANGE), 2))
        ax.set_xticklabels(label for id, label in enumerate(SAMPLES_PER_ITERATION_RANGE) if id % 2 == 0)
        ax.set_yticks(range(0, ITERATIONS, 5))
        ax.set_yticklabels(range(1, ITERATIONS + 1, 5))
        ax.set_title(f"{threshold}%")
        ax.grid(False)
    fig.colorbar(color_scalar, ax=axs, shrink=0.5)
    axs[0].set_ylabel("Iterations")
    # fig.supxlabel("Samples per iteration")
    # fig.supylabel("Iterations")
    axs[2].set_xlabel("Samples per iteration")

    # return fig

In [ ]:
multi_implot(classified)

In [ ]:
multi_implot(classified_rmad)


In [ ]:
instruction_rciw = rciw.join(actual_instr, on='bench', how='inner')
instruction_rciw

In [ ]:
# instruction_sample_rciw = instruction_rciw.copy(deep=True).reset_index()
# instruction_sample_rciw['num_samples'] = instruction_sample_rciw['iterations'] * instruction_sample_rciw['samples']
# total_samples = total_samples.sort_values(['rciw']).drop_duplicates(['bench', 'num_samples'],keep='last')# instruction_rciw = rciw.join(actual_instr, on='bench', how='inner')

# sns.scatterplot(instruction_sample_rciw, x='instructions', y='rciw', hue='num_samples')
# plt.yscale('log')
# plt.legend(loc='upper center')

# iters = range(1,31)
iters = range(5, 31, 5)
fig, axs = plt.subplots(2, 3, sharex=True, sharey=True)

for idx, s in enumerate(iters):
    ax = axs.flatten()[idx]
    sns.scatterplot(instruction_rciw.loc[:, s, :], x='instructions', y='rciw', ax=ax, alpha=0.5)
    ax.set_title(f"{s} iterations")
    ax.set_ylabel("RCIW (%)")
    ax.set_xlabel("Instructions")
    # ax.set_ylim([0, 20])
plt.tight_layout()

In [ ]:
coeff = []
for iteration in range(ITERATIONS):
    for samples in SAMPLES_PER_ITERATION_RANGE:
        # print()
        selection = instruction_rciw.loc[slice(None), iteration + 1, samples]
        # print(selection)
        # pearson = np.corrcoef(selection['rciw'], selection['instructions'])[0][1]
        pearson = stats.spearmanr(selection['rciw'], selection['instructions']).statistic
        coeff.append(pearson)
coeff = np.array(coeff).reshape(ITERATIONS, len(SAMPLES_PER_ITERATION_RANGE))
plt.xlabel("Samples per iteration")
plt.xticks(range(0, len(SAMPLES_PER_ITERATION_RANGE), 1),
           [label for id, label in enumerate(SAMPLES_PER_ITERATION_RANGE) if id % 1 == 0])
plt.imshow(coeff, cmap='RdYlGn', aspect='auto', vmin=-1, vmax=1)
plt.yticks(range(0, ITERATIONS, 5), range(1, ITERATIONS + 1, 5))
plt.colorbar()
plt.grid(False)
print(f"min {coeff.min():.3}; med: {np.median(coeff):.3}; max: {coeff.max():.3}")

In [ ]:
# iters = range(1,31)
iters = range(5, 31, 5)
fig, axs = plt.subplots(2, 3, sharex=True, sharey=True)
# local = rciw.loc[(slice(None), i, slice(None))]
ll = rciw.reset_index(level='samples')
ll['rciw'] = ll['rciw'] * 100
for idx, s in enumerate(iters):
    ax = axs.flatten()[idx]
    sns.lineplot(ll.loc[(slice(None), s, slice(None))], x='samples', y='rciw', units='bench', hue='bench', legend=False,
                 ax=ax, estimator=None)
    ax.set_title(f"{s} iterations")
    ax.set_ylabel("RCIW (%)")
    ax.set_xlabel("Samples per iteration")
    ax.set_ylim([0, 20])
plt.tight_layout()

In [ ]:
rciw.reset_index()[(rciw.reset_index()['rciw'] > .10) & (rciw.reset_index()['iterations'] > 4)].groupby('bench').count()

In [ ]:
# iters = range(1,31)
iters = range(5, 31, 5)
fig, axs = plt.subplots(2, 3, sharex=True, sharey=True)
# local = rciw.loc[(slice(None), i, slice(None))]

for idx, s in enumerate(iters):
    ax = axs.flatten()[idx]
    sns.lineplot(rmad.loc[(slice(None), s, slice(None))], x='samples', y='rmad', units='bench', hue='bench',
                 legend=False, ax=ax, estimator=None)
    ax.set_title(f"{s} iterations")
    ax.set_xlabel("Samples per iteration")
    # ax.set_ylim([0, 1.5])
plt.tight_layout()
# plt.yscale('log')

In [ ]:
# iters = range(1,31)
import math

samps = range(50, 301, 50)
fig, axs = plt.subplots(2, math.floor(len(samps) / 2), sharex=True, sharey=True)
# local = rciw.loc[(slice(None), i, slice(None))]

for idx, s in enumerate(samps):
    print(f"{idx}. {s}")
    ax = axs.flatten()[idx]
    sns.lineplot(rciw.loc[(slice(None), slice(None), s)], x='iterations', y='rciw', units='bench', hue='bench',
                 legend=False, ax=ax, estimator=None)
    ax.set_title(f"{s} samples")
    ax.set_xlabel("Number of iterations")
    # ax.set_ylim([0, 1.5])
plt.tight_layout()
# plt.yscale('log')

In [ ]:
ll = rciw.reset_index(level='samples')
# ll['rciw'] = ll['rciw'] * 1000
ll['rciw'] *= 100
regular_medians = ll.groupby(['iterations', 'samples']).median()
hj_medians = ll.groupby(['iterations', 'samples']).apply(lambda df: med_hj(df['rciw'])[0],
                                                         include_groups=False).reset_index().rename(columns={0: 'rciw'})
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
sns.lineplot(hj_medians, x='samples', y='rciw', units='iterations', hue='iterations', legend=True, palette='viridis_r',
             estimator=None, ax=axs[0])
sns.lineplot(hj_medians, x='iterations', y='rciw', units='samples', hue='samples', legend=True, palette='viridis_r',
             estimator=None, ax=axs[1])

axs[0].set_xlabel("Samples taken per iteration")
axs[1].set_ylabel("RCIW (%)")
axs[0].set_ylabel("RCIW (%)")
axs[1].set_xlabel("Number of iterations used")


def is_monotonic_decreasing(data) -> bool:
    return not not (np.diff(data) < 0).all()

ll
# [is_monotonic_decreasing(group[group['iterations'] > 15]['rciw']) for  label, group in hj_medians.groupby('samples')]# ll.groupby(['iterations','samples']).median()
# [is_monotonic_decreasing(group['rciw']) for  label, group in hj_medians.groupby('iterations')]# ll.groupby(['iterations','samples']).median()

In [ ]:
for label, group in hj_medians.groupby('iterations'):
    d = np.polynomial.polynomial.polyfit(x=group['samples'], y=group['rciw'], deg=5)
    sns.lineplot(x=SAMPLES_PER_ITERATION_RANGE,
                 y=np.polynomial.polynomial.polyval(list(SAMPLES_PER_ITERATION_RANGE), d))

In [ ]:
def func(x, a, b, c, d):
    return np.log((x + d) / a) / b + c


fitting_data = hj_medians[hj_medians['iterations'] > 4]
fig, axs = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
sns.lineplot(fitting_data, x='samples', y='rciw', units='iterations', hue='iterations', legend=True,
             palette='viridis_r', estimator=None, ax=axs[1])

SSSE = 0
popts = []
from scipy.optimize import curve_fit

for label, group in fitting_data.groupby('iterations'):
    (fit, pcov, info, mesg, ier) = curve_fit(func, xdata=group['samples'], ydata=group['rciw'], full_output=True,
                                             bounds=([0, -np.inf, -np.inf, -21], [np.inf, 0, np.inf, np.inf]))
    sns.lineplot(x=SAMPLES_PER_ITERATION_RANGE, y=map(lambda x: func(x, *fit), SAMPLES_PER_ITERATION_RANGE), ax=axs[0],
                 legend=False, alpha=0.3)
    perr = np.sqrt(np.diag(pcov))
    # print(perr)
    popts.append(fit)
    SSE = np.power(info['fvec'], 2).sum()
    SSSE += SSE
    print(label, ",", ",".join(map(str, fit)))
print(SSSE)

# print(list(map(lambda x: func(x, *fit), SAMPLES_PER_ITERATION_RANGE)))

In [ ]:
for opt in np.array(popts).T:
    sns.lineplot(x=np.arange(5,31), y=opt)
    
plt.legend(['a', 'b', 'c', 'd'])

In [ ]:

popt_fit = np.polynomial.polynomial.polyfit(np.arange(5, 31), np.array(popts), deg=1).T
print(popt_fit)
params = ['a', 'b', 'c', 'd']
for idx, param in enumerate(popt_fit):
    print(f"{params[idx]}: v = {param[1]} * iteration {param[0]:+}")
    print(f"mean {params[idx]}: {(np.arange(5, 31) * param[1] + param[0]).mean()}")
# np.array(popts)



In [ ]:
predictor = lambda sample, iteration: func(sample, *[1.1497813734895757,
                                          -0.56510988553921 * iteration - 10.021575835328116,
                                          -0.01411705944972687 * iteration +0.6352021390905219,
                                           -19.41007111388554])
vec_func = np.vectorize(predictor)
f, axs = plt.subplots(5, 5, sharex=True, sharey=True, squeeze=True, figsize=(10,10))
axs = axs.flatten()
for i in range(5, 30):
    sns.lineplot(x=np.arange(25, 300), y=vec_func(np.arange(25, 300), i), label='prediction', ax=axs[i-5])
    sns.lineplot(hj_medians.iloc[hj_medians.groupby('iterations').indices.get(i)], x='samples', y='rciw', label='actual',ax=axs[i-5], )
    axs[i-5].set_title(i)
    # scipy.stats.linregress(hj_medians)
